In [18]:
##################################################################################################
# Patch-based Classification of Breast Cancer Histology Images using CNNs
# LE48: MiniProject
# Jan Ondras (jo356), Trinity College
# 2017/2018
##################################################################################################
##################################################################################
# RCNN version, train
##################################################################################

#import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import os
from time import time
from math import ceil
# from PIL import Image

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras import metrics
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Input, TimeDistributed, LSTM
from keras.utils import multi_gpu_model
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Model
import keras

seed = 7 #7
np.random.seed(seed)

# seeds = [7, 34, 888, 456456, 33, 16947, 7764, 3396, 20500, 3009467] # 10 random seeds = 10 rand restarts

# for seed_index, seed in enumerate(seeds): # 10 random restarts
#     np.random.seed(seed)
#     if seed == 7 or seed == 34:        # TO DELETE
#         continue
######################################################################################
# TO SET
######################################################################################
overwrite = True
# overwrite = False

target_size = (256,256)
model_ID = 'x00' # + str(seed_index)
patch_width = 256                     # 256, 512, 1024
patch_height = patch_width
patch_stride = patch_width                  
model_type = 'RCNN_' + str(patch_width) + '_' + str(patch_stride) + '_' + model_ID

RGB_means = [
    180.375933345 , 148.276127037 , 174.592562423
] 
# AWS: 180.375933345 , 148.276127037 , 174.592562423
# HPC: 180.7795009644826 , 148.3537065177495 , 174.59451631969864
######################################################################################
# Create model directory for model checkpointing
if overwrite == False:
    if os.path.exists('./../Model/' + model_type):
        raise NameError('Model directory ' + './../Model/' + model_type + ' already exists!')
    else:
        os.mkdir('./../Model/' + model_type)
else:
    if not os.path.exists('./../Model/' + model_type):
        os.mkdir('./../Model/' + model_type)
model_checkpoint_path_prefix = './../Model/' + model_type + '/' + model_type + '_'
print model_type

# History file for checkpointing loss, acc and time per epoch
history_filename = './../History/history_' + model_type + '.txt'
if overwrite == False and os.path.exists(history_filename):
        raise NameError('History file ' + history_filename + ' already exists!')
with open(history_filename, 'w') as f:
    f.write("")

classes = ['Normal', 'Benign', 'InSitu', 'Invasive'] # correspond to labels 0,1,2,3 in this order

# Image size (assume same for all images)
img_width =  2048
img_height = 1536
img_size = (img_height, img_width)
pix_scale = 0.42 # micrometers

# Number of examples (whole images) per class before data augmentation
N_imgs_per_class = 100
train_valid_split = 0.75
N_train_samples = 300           # in terms of #images
N_validation_samples = 100

N_patches_x = (((img_width - patch_width) / patch_stride) + 1)
N_patches_y = (((img_height - patch_height) / patch_stride) + 1)
N_patches_per_img = N_patches_x * N_patches_y
print "Number of patches per image: ", N_patches_per_img

path_prefix = './../Dataset/ICIAR2018_BACH_Challenge/Photos_SN_split'
train_data_dir = path_prefix + '/train'
validation_data_dir = path_prefix + '/validation'

epochs = 1000
# Batch sizes in terms of number of images;
train_IMG_batch_size = 5
evaluate_IMG_batch_size = 5

print "Batch size: ", train_IMG_batch_size
print "Evaluate batch size: ", evaluate_IMG_batch_size

# Input into model - these are resized patches 256x256; as timeseries of N_patches_per_img
if K.image_data_format() == 'channels_first':
    input_shape = (3, N_patches_per_img, target_size[0], target_size[1])
    print "Channels first"
else:
    input_shape = (N_patches_per_img, target_size[0], target_size[1], 3) # this one
    print "Channels last"

def substract_mean(img):
    # RGB
    img[:,:,0] -= RGB_means[0]
    img[:,:,1] -= RGB_means[1]
    img[:,:,2] -= RGB_means[2]    
    return img

# def img_to_patches(img):
#     '''
#     Input = (1536, 2048, 3)
#     Output = (N_patches_per_img, 256, 256, 3)
#     '''
#     patches = np.zeros( (N_patches_per_img, target_size[0], target_size[1], 3) )
#     patches = []
#     patch_ID = 0
#     for i in range(N_patches_x):
#         for j in range(N_patches_y):
#             p = img[j*patch_stride:j*patch_stride + patch_height, i*patch_stride:i*patch_stride + patch_width, :]
#             # Resize to target_size
#             patches[patchID] = p
#             #patches.append( img[j*patch_stride:j*patch_stride + patch_height, i*patch_stride:i*patch_stride + patch_width, :] )
            
#             patch_ID += 1
#     if patch_ID != N_patches_per_img:
#         raise ValueError("Incorrect number of patches per image!")
        
#     return patches
#     #return np.array(patches)
    
def N_imgs_to_patches(imgs):
    '''
    Input = (train_IMG_batch_size, 1536, 2048, 3)
    Output = (train_IMG_batch_size, N_patches_per_img, 256, 256, 3)
    '''
    patches = np.zeros( (train_IMG_batch_size, N_patches_per_img, target_size[0], target_size[1], 3) )
    # Iterate over images
    for imID in range(imgs.shape[0]):
    
        # Iterate over patches
        patchID = 0
        for i in range(N_patches_x):
            for j in range(N_patches_y):
                p = imgs[imID, j*patch_stride:j*patch_stride + patch_height, i*patch_stride:i*patch_stride + patch_width, :]
                #print p.shape
                # Resize to target_size TODOOOOOOOOO
                #Image.fromarray(p).resize(target_size).getdata()
                p = cv2.resize(p, target_size, interpolation=cv2.INTER_NEAREST)
                patches[imID, patchID] = p
                #patches.append( img[j*patch_stride:j*patch_stride + patch_height, i*patch_stride:i*patch_stride + patch_width, :] )

                patchID += 1
        if patchID != N_patches_per_img:
            raise ValueError("Incorrect number of patches per image!")

    return patches
    #return np.array(patches)

# N_imgs_to_patches = np.vectorize(img_to_patches)

######################################################################################
# SETUP MODEL
######################################################################################

# model = Sequential([ # ORIGINAL
#     TimeDistributed(Conv2D(16, (3, 3), strides=(1, 1), padding='same', activation='relu'), input_shape=input_shape), 
#     TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')), 
#     TimeDistributed(Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu')),
#     TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')),
#     TimeDistributed(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')),
#     TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')),
#     TimeDistributed(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')),
#     TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')),
#     TimeDistributed(Conv2D(32, (3, 3), strides=(1, 1), padding='same', activation='relu')),
#     TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')),
    
#     TimeDistributed(Flatten()), #------------------------------ has dimensionality 32
#     LSTM(32, return_sequences=False),
#     Dense(128, activation='relu'),
#     Dense(64, activation='relu'),
#     Dense(4, activation='softmax')
# ])

# # From VGG
# model = Sequential([ # ORIGINAL
#     TimeDistributed(Conv2D(16, (3, 3), activation='relu', padding='same', name='block1_conv1'), input_shape=input_shape), 
#     TimeDistributed(Conv2D(16, (3, 3), activation='relu', padding='same', name='block1_conv2')), 
#     TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool')),
    
#     TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same', name='block2_conv1')),
#     TimeDistributed(Conv2D(32, (3, 3), activation='relu', padding='same', name='block2_conv2')),
#     TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool')),
    
#     TimeDistributed(Conv2D(16, (3, 3), activation='relu', padding='same', name='block3_conv1')),
#     TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')),
    
#     TimeDistributed(Conv2D(16, (3, 3), activation='relu', padding='same', name='block4_conv1')),
#     TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool')),
    
#     TimeDistributed(Conv2D(16, (3, 3), activation='relu', padding='same', name='block5_conv1')),
#     TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool')),
    
#     TimeDistributed(Conv2D(16, (3, 3), activation='relu', padding='same', name='block5_conv1')),
#     TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2), name='block6_pool')),
    
#     TimeDistributed(Conv2D(16, (3, 3), activation='relu', padding='same', name='block5_conv1')),
#     TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2), name='block6_pool')),
    
# #     TimeDistributed(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1')),
# #     TimeDistributed(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2')),
# #     TimeDistributed(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool')),
    
#     TimeDistributed(Flatten()), #------------------------------ has dimensionality 32
#     LSTM(64, return_sequences=False),
#     Dense(256, activation='relu'),
#     Dense(128, activation='relu'),
#     Dense(4, activation='softmax')
# ])



# # Experiemnetnst    0,1,2
# model = Sequential([ # ORIGINAL
#     TimeDistributed(Conv2D(16, (3, 3), strides=(1, 1), padding='valid', activation='relu'), input_shape=input_shape), 
#     TimeDistributed(MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid')), 
#     TimeDistributed(Conv2D(32, (3, 3), strides=(1, 1), padding='valid', activation='relu')),
#     TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')),
#     TimeDistributed(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')),
#     TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')),
#     TimeDistributed(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')),
#     TimeDistributed(MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid')),
#     TimeDistributed(Conv2D(32, (3, 3), strides=(1, 1), padding='valid', activation='relu')),
#     TimeDistributed(MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid')),
    
#     TimeDistributed(Flatten()), #------------------------------ has dimensionality 32
#     LSTM(64, return_sequences=False),
#     Dense(256, activation='relu'),
#     Dense(128, activation='relu'),
#     Dense(4, activation='softmax')
# ])




# model = Sequential([
#     TimeDistributed(Conv2D(16, (3, 3), strides=(1, 1), padding='valid', activation='relu'), input_shape=input_shape), 
#     TimeDistributed(MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid')), 
#     TimeDistributed(Conv2D(32, (3, 3), strides=(1, 1), padding='valid', activation='relu')),
#     TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')),
#     TimeDistributed(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')),
#     TimeDistributed(MaxPooling2D(pool_size=(2, 2), strides=None, padding='valid')),
#     TimeDistributed(Conv2D(64, (3, 3), strides=(1, 1), padding='same', activation='relu')),
#     TimeDistributed(MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid')),
#     TimeDistributed(Conv2D(32, (3, 3), strides=(1, 1), padding='valid', activation='relu')),
#     TimeDistributed(MaxPooling2D(pool_size=(3, 3), strides=None, padding='valid')),
    
#     TimeDistributed(Flatten()), #------------------------------ has dimensionality 32
#     LSTM(256, dropout=0.5, return_sequences=False),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(4, activation='softmax')
# ])
print (model.summary())
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(), # 'adadelta' rmsprop adam
              metrics=['accuracy'])

# Train and validation generators load whole images
train_datagen = ImageDataGenerator(
    preprocessing_function=substract_mean, 
    rotation_range=180, 
    horizontal_flip=True, 
    fill_mode='reflect' # reflect / wrap / nearest
) 
test_datagen = ImageDataGenerator(
    preprocessing_function=substract_mean
)
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size= img_size,
    batch_size= train_IMG_batch_size,
    class_mode='categorical',
    classes=classes)
validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size= img_size,
    batch_size= evaluate_IMG_batch_size,
    class_mode='categorical',
    classes=classes)

######################################################################################
# TRAINING & VALIDATION
######################################################################################

max_val_acc = 0.
epochs_wo_improvement = 0
patience = 10

for e in range(1, epochs+1):
    print "Epoch {:03d}".format(e)
    st = time()
    
    train_res = np.zeros(2)
    N_batches = 0.
    # Get batch of training images
    for X_IMG_batch, y_batch in train_generator:
        N_batches += 1.
        
        #print N_batches, X_IMG_batch.shape, y_batch.shape
    
        # Generate patches for each image
        #X_PATCH_batch = N_imgs_to_patches(X_IMG_batch) # this now has shape (train_IMG_batch_size, N_patches_per_img, 256, 256, 3)

        X_PATCH_batch = N_imgs_to_patches(X_IMG_batch)  #np.array( map(img_to_patches, X_IMG_batch) )
        
        #h = model.fit(X_PATCH_batch, y_batch, batch_size=train_IMG_batch_size)
        r = model.train_on_batch(X_PATCH_batch, y_batch)
        train_res = train_res + r
        #print r
        
        #print h
        
        # Stop generator if all training examples were seen in this epoch
        if N_batches * train_IMG_batch_size >= N_train_samples:
            break
    train_res = train_res / N_batches
    print "\tTrain:", train_res
    #print "#train batches =", N_batches
    
    N_batches = 0.
    val_res = np.zeros(2)
    # Get batch of validation images
    for X_IMG_batch, y_batch in validation_generator:
        N_batches += 1.

        # Generate patches for each image
        X_PATCH_batch = N_imgs_to_patches(X_IMG_batch)  #np.array( map(img_to_patches, X_IMG_batch) )

        #r = model.evaluate(X_PATCH_batch, y_batch, batch_size=evaluate_IMG_batch_size)
        r = model.test_on_batch(X_PATCH_batch, y_batch)
        val_res = val_res + r

        # Stop generator if all training examples were seen in this epoch
        if N_batches * evaluate_IMG_batch_size >= N_validation_samples:
            break
    val_res = val_res / N_batches
    #print "#val batches =", N_batches

    tpe = time()-st
    print "\tValid:", val_res
    print "\tTime:", tpe
    
    
    
    # Save model and results
    model.save(model_checkpoint_path_prefix + '{:03d}'.format(e)+'.hdf5')
    
    with open(history_filename, 'a') as f:
        f.write( str(train_res[0]) + "," + str(train_res[1]) + "," 
               + str(val_res[0]) + "," + str(val_res[1]) + "," + str(tpe) + "\n")

    # Early stopping
    if val_res[1] < max_val_acc:
        epochs_wo_improvement += 1
    else:
        epochs_wo_improvement = 0
        max_val_acc = val_res[1]
    
    if epochs_wo_improvement >= patience:
        print "Early stopping ... patience =", patience
        break

RCNN_256_256_x00
Number of patches per image:  48
Batch size:  10
Evaluate batch size:  10
Channels last
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_172 (TimeDi (None, 48, 254, 254, 16)  448       
_________________________________________________________________
time_distributed_173 (TimeDi (None, 48, 84, 84, 16)    0         
_________________________________________________________________
time_distributed_174 (TimeDi (None, 48, 82, 82, 32)    4640      
_________________________________________________________________
time_distributed_175 (TimeDi (None, 48, 41, 41, 32)    0         
_________________________________________________________________
time_distributed_176 (TimeDi (None, 48, 41, 41, 64)    18496     
_________________________________________________________________
time_distributed_177 (TimeDi (None, 48, 20, 20, 64)    0         
_____________________________________

ResourceExhaustedError: OOM when allocating tensor with shape[480,254,254,16]
	 [[Node: time_distributed_172/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](time_distributed_172/Reshape, time_distributed_172/kernel/read)]]
	 [[Node: metrics_16/acc/Mean/_3271 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_2576_metrics_16/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op u'time_distributed_172/convolution', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/runpy.py", line 72, in _run_code


In [1]:
import cv2


In [ ]:
#########################
# OLD, not used

##############################################################################
# Plot training and validation loss and accuracy curves
# Plot training times per epoch
##############################################################################
from matplotlib import pyplot as plt

x = range(1, len(model_hist.times)+1)

# Loss Curves
plt.figure(figsize=[8,6])
plt.plot(x, model_hist.train_loss,'ro-')#,linewidth=2.0)
plt.plot(x, model_hist.val_loss,'bo-')#,linewidth=2.0)
plt.legend(['Training loss', 'Validation Loss'])#,fontsize=18)
plt.xticks(x, x)
plt.xlabel('Epoch')#,fontsize=16)
plt.ylabel('Loss')#,fontsize=16)
# plt.title('Loss Curves',fontsize=16)
 
# Accuracy Curves
plt.figure(figsize=[8,6])
plt.plot(x, model_hist.train_acc,'ro-')#,linewidth=2.0)
plt.plot(x, model_hist.val_acc,'bo-')#,linewidth=2.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'])#,fontsize=18)
plt.xticks(x, x)
plt.xlabel('Epoch')#,fontsize=16)
plt.ylabel('Accuracy')#,fontsize=16)
# plt.title('Accuracy Curves',fontsize=16)
plt.show()

# Training time
plt.figure(figsize=[8,6])
plt.plot(x, np.array(model_hist.times)/60., 'o-')
plt.xticks(x, x)
plt.xlabel('Epoch')
plt.ylabel('Training time per epoch (min)')
plt.legend(['Mean = '+str(np.mean(model_hist.times)/60.)+" min"])
plt.show()

# Oly if whole training completes ...
# # Loss Curves
# plt.figure(figsize=[8,6])
# plt.plot(history.history['loss'],'r',linewidth=2.0)
# plt.plot(history.history['val_loss'],'b',linewidth=2.0)
# plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
# plt.xlabel('Epochs ',fontsize=16)
# plt.ylabel('Loss',fontsize=16)
# plt.title('Loss Curves',fontsize=16)
# # Accuracy Curves
# plt.figure(figsize=[8,6])
# plt.plot(history.history['acc'],'r',linewidth=2.0)
# plt.plot(history.history['val_acc'],'b',linewidth=2.0)
# plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
# plt.xlabel('Epochs ',fontsize=16)
# plt.ylabel('Accuracy',fontsize=16)
# plt.title('Accuracy Curves',fontsize=16)
# plt.show()

In [ ]:

    # just to check
    # print train_generator.class_indices, train_generator.classes
    # print validation_generator.class_indices, validation_generator.classes

    # early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    # model.fit(x, y, validation_split=0.2, callbacks=[early_stopping])

    # Reduce learning rate when val_loss stopped improving
    # reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
    #                               patience=5, min_lr=0.001)
    # model.fit(X_train, Y_train, callbacks=[reduce_lr])
    # or: keras.callbacks.LearningRateScheduler(schedule)

# Custom generator
# https://www.kaggle.com/sinkie/keras-data-augmentation-with-multiple-inputs
# def generate_arrays_from_file(path):
#     while 1:
#         f = open(path)
#         for line in f:
#             # create numpy arrays of input data
#             # and labels, from each line in the file
#             x1, x2, y = process_line(line)
#             yield ({'input_1': x1, 'input_2': x2}, {'output': y})
#         f.close()

# model.fit_generator(generate_arrays_from_file('/my_file.txt'),
#                     steps_per_epoch=10000, epochs=10)